In [ ]:
import numpy as np
import pandas as pd

# Unit Tests

## Overview and Principles
Testing is the process by which you exercise your code to determine if it performs as expected. The code you are testing is referred to as the **code under test**. 

There are two parts to writing tests.

The collection of tests performed are referred to as the **test cases**. The fraction of the code under test that is executed as a result of running the test cases is referred to as **test coverage**.

For dynamical languages such as Python, it's extremely important to have a high test coverage. In fact, you should try to get 100% coverage.

Why is test coverage is important in a dynamic language like Python?

Test cases can be of several types. Some common classifications of test cases are:

Another principle of testing is to limit what is done in a single test case. Generally, a test case should focus on **one use of one function**. Sometimes, this is a challenge since the function being tested may call other functions that you are testing. This means that bugs in the called functions may cause failures in the tests of the calling functions. Often, you sort this out by knowing the structure of the code and focusing first on failures in lower level tests. In other situations, you may use more advanced techniques called *mocking*. A discussion of mocking is beyond the scope of this lecture.

## Test-driven development

A best practice is to develop your tests while you are developing your code. Indeed, one school of thought in software engineering, called **test-driven development**, advocates that you write the tests *before* you implement the code under test so that the test cases become a kind of specification for what the code under test should do.

**This is how you should approach development going forward in this course.** Write your tests first.  They all fail.  Write the code for the functions to make the tests pass.

## Examples of Test Cases
This section presents examples of test cases. The code under test is the calculation of entropy.

### Entropy of a set of probabilities
$$
H = -\sum_i p_i \log(p_i)
$$
where $\sum_i p_i = 1$.

In [ ]:
import numbers

def entropy(p):
    if any([not isinstance(p_i, numbers.Number) for p_i in p]):
        raise ValueError("At least one input is not a number")
    if any([(p_i < 0.0) or (p_i > 1.0) for p_i in p]):
        raise ValueError("At least one input is out of range [0...1]")
    elif not np.isclose(1, np.sum(p), atol=1e-08):
        raise ValueError("The list of input probabilities does not sum to 1")
    else:
        pass
    
    items = []
    for p_i in p:
        if p_i > 0:
            interm = p_i * np.log2(p_i)
            items.append(interm)
    return np.abs(np.sum(items))

### One shot test

We know from previous discussions that when we have 4 states and they are all equally likely, the number of bits required should be 2.

In [ ]:
entropy([.25, .25, .25, .25])

Another example is that the entropy of a random variable when there is only one possible outcome is 0, therefore:

In [ ]:
entropy([1])

Suppose that all of the probability of a distribution is at one point. An example of this is a coin with two heads. Whenever you flip it, you always get heads. That is, the probability of a head is 1.

What is the entropy of such a distribution? From the calculation above, we see that the entropy should be $log(1)$, which is 0. This means that we have a test case where we know the result!

Let's write a test for the 1-probability case:

Can we make it so that the test is more *generic*? As in, it is easy to add a new one-shot test case?

**NEAT!** We can use this structure to do a bunch of these types all at once, e.g.

**Question**: What is an example of another one-shot test? (Hint: You need to know the expected result.)

### Edge tests

One edge test of interest is to provide an input that is *not* a distribution in that probabilities don't sum to 1.  These should generate an exception of type ValueError

Another edge test is when we pass a probability that is out of range.

Let's try writing a function to test this edge case.

#### Important note for edge tests that raise exceptions!

You often have to write your tests using `try` and `except` blocks being sure to catch the correct Exception type, e.g.

In [ ]:
def test_entropy_sum_to_1():
    # TODO: implement
    pass

def test_entropy_negative_probability():
    # TODO: implement
    pass

In [ ]:
test_entropy_sum_to_1()
test_entropy_negative_probability()

Now let's consider a pattern test. Examining the structure of the calculation of $H$, we consider a situation in which there are $n$ equal probabilities. That is, $p_i = \frac{1}{n}$.
$$
H = -\sum_{i=1}^{n} p_i \log(p_i) 
= -\sum_{i=1}^{n} \frac{1}{n} \log(\frac{1}{n}) 
= n (-\frac{1}{n} \log(\frac{1}{n}) )
= -\log(\frac{1}{n})
$$
For example, entropy([0.5, 0.5]) should be $-log(0.5)$.

Let's write a pattern test for this pattern:

In [ ]:
# Pattern test
def test_equal_probabilities(n):
    # TODO: implement
    pass
        
# Run a test
for n in range(1,100):
    test_equal_probabilities(n)

You see that there are many, many cases to test. So far, we've been writing special code for each test case. We can do better.

## Testing Data Producing Code
Much of your python (or R) code will be creating and/or transforming dataframes. A dataframe is structured like a table with:

- Columns that have values of the same type
- Rows that have a value for each column
- An index that uniquely identifies a row.

In [ ]:
def makeProbabilityMatrix(column_names, nrows):
    """
    Makes a dataframe with the specified column names such that each
    cell is a value in [0, 1] and columns sum to 1.
    :param list-str column_names: names of the columns
    :param int nrows: number of rows
    """
    df = pd.DataFrame(np.random.uniform(0, 1, (nrows, len(column_names))))
    df.columns = column_names
    for column in df.columns:
        df[column] = df[column]/df[column].sum()
    return df
                      

Example call and result:

In [ ]:
makeProbabilityMatrix(['a', 'b'], 3)

In [ ]:
# Test: Check columns
columns = ['a', 'b']
df = makeProbabilityMatrix(columns, 3)
set(columns) == set(df.columns)

### Exercise
Write a function that tests the following:
- The returned dataframe has the expected columns
- The returned dataframe has the expected rows
- Values in columns are of the correct type and range
- Values in column sum to 1

## Unittest Infrastructure

There are several reasons to use a test infrastructure:
- If you have many test cases (which you should!), the test infrastructure will save you from writing a lot of code.
- The infrastructure provides a uniform way to report test results, and to handle test failures.
- A test infrastructure can tell you about coverage so you know what tests to add.

[We'll be using the `unittest` framework](https://docs.python.org/3/library/unittest.html). This is a separate Python package. Using this infrastructure, requires the following:
1. import the unittest module
1. define a class that inherits from unittest.TestCase
1. write methods that run the code to be tested and check the outcomes.

The last item has two subparts. First, we must identify which methods in the class inheriting from unittest.TestCase are tests. You indicate that a method is to be run as a test by having the method name begin with "test".

Second, the "test methods" should communicate with the infrastructure the results of evaluating output from the code under test. This is done by using `assert` statements. For example, `self.assertEqual` takes two arguments. If these are objects for which `==` returns `True`, then the test passes. Otherwise, the test fails.

In [ ]:
import unittest

# Define a class in which the tests will run
class UnitTests(unittest.TestCase):

    # Each method in the class to execute a test
    def test_success(self):
        self.assertEqual(1, 1)
        
    def test_success1(self):
        self.assertTrue(1 == 1)

    def test_failure(self):
        self.assertLess(1, 2)
 
suite = unittest.TestLoader().loadTestsFromTestCase(UnitTests)
_ = unittest.TextTestRunner().run(suite)


**Code for homework or your work should use test files.** In this lesson, we'll show how to write test codes in a Jupyter notebook. This is done for pedidogical reasons. It is **NOT** not something you should do in practice, except as an intermediate exploratory approach. 

### Exercise: unittest
- Rewrite the above one-shot tests for entropy of equal probabilities using the unittest infrastructure. Notice the *nested function* called `test`. The test below is actually a **pattern test**, a kind of extended one-shot test. If you just want to test a single case (ie not the full pattern, just a particular count), then you wouldn't need the nested function.

In [ ]:
# Implementating a pattern test. Use functions in the test.
import unittest

# Define a class in which the tests will run
class TestEntropy(unittest.TestCase):
        
    def test_equal_probability(self):
        def test(count):
            """
            Invokes the entropy function for a number of values equal to count
            that have the same probability.
            :param int count:
            """
            pass

        test(2)
        test(20)
        test(200)

suite = unittest.TestLoader().loadTestsFromTestCase(TestEntropy)
_ = unittest.TextTestRunner().run(suite)

## Testing For Exceptions

Edge test cases often involves handling exceptions. One approach is to code this directly.

In [ ]:
import unittest

# Define a class in which the tests will run
class TestEntropy(unittest.TestCase):
        
    def test_invalid_probability(self):
        try:
            entropy([0.1, 0.5])
            # This isn't epecially pretty, but it works.
            self.assertTrue(False)
        except ValueError:
            self.assertTrue(True)

suite = unittest.TestLoader().loadTestsFromTestCase(TestEntropy)
_ = unittest.TextTestRunner().run(suite)

`unittest` provides help with testing exceptions.

In [ ]:
import unittest

# Define a class in which the tests will run
class TestEntropy(unittest.TestCase):
        
    def test_invalid_probability(self):
        with self.assertRaises(ValueError):
            entropy([0.1, 0.5])
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestEntropy)
_ = unittest.TextTestRunner().run(suite)

### Exercise: A Full Test Suite

* Write a full test suite for the entropy function, including edge tests.

In [ ]:
import unittest

# Define a class in which the tests will run
class TestEntropy(unittest.TestCase):
  """Write the full set of tests."""

## Discussion
**Question**: What tests would you write for a plotting function?

**Question**: How would you test functions with side effects, like file operations or printing?

## Test Files

Although I presented the elements of `unittest` in a notebook. **your tests should be in a file**. If the name of module with the code under test is `foo.py`, then the name of the test file should be `test_foo.py`.

The structure of the test file will be very similar to cells above. You will import `unittest`. You must also import the module with the code under test. We usually put the tests in a folder called `tests`.

In order for the tests to run when you run the test file on the command line, you should include a main block (like we learned last week) that uses the `unittest` module's main to run all tests.

```python
if __name__ == '__main__':
    unittest.main()
```

You will NOT include the `TestLoader`/`TestRunner` lines that we've been using in Jupyter Notebooks.

Lastly, you should include an empty file called `__init__.py` in the `tests` directory. We will discuss in a later lecture what this file is (it has to do with packages!).

You can then run the tests one of two ways:

1. You can run the test directly with Python:
  ```bash
  $ python -m tests.test_module_name
  ```
1. You can use "auto discovery" and let the framework automatically find all of the tests.
  ```bash
  $ python -m unittest discover
  ```

### Exercise: A Test Suite In A File

1. Move the entropy function into a file called `entropy.py` in a folder called `entropy` and the test suite for it into a file called `test_entropy.py`, which is located in the `tests` directory. Make sure you can run the test suite.

```
my_project
    my_module
        my_module.py
    tests
        test_my_module.py
```

## Exercise: Test Driven Development

Start by writing the tests. Then write the code.

We illustrate this by considering a function geomean that takes a list of numbers as input and produces the geometric mean on output.

The geometric mean is defined as the nth root of the product of n numbers.

In [ ]:
import unittest

# Define a class in which the tests will run
class TestGeomean(unittest.TestCase):
    def test():
        pass

    # Edge tests
    # One-shot tests
    # Pattern tests

suite = unittest.TestLoader().loadTestsFromTestCase(TestEntropy)
_ = unittest.TextTestRunner().run(suite)

In [ ]:
#def geomean(argument?):
#    return ?

## Other infrastructures
- pytest
- nose
- Use binary functions that being with "test"

## References

https://www.youtube.com/watch?v=GEqM9uJi64Q (Pydata 2015)
https://www.youtube.com/watch?v=yACtdj1_IxE (Pycon 2017)

The first talk mentions some packages:
engarde - https://github.com/TomAugspurger/engarde
Hypothesis - https://hypothesis.readthedocs.io/en/latest/
Feature Forge - https://github.com/machinalis/featureforge


Detlef Nauck talk: 
http://ukkdd.org.uk/2017/info/talks/nauck.pdf
He also had a list of R tools but I could not find the slides form the talk I saw.

Test Driven Data Analysis:
https://www.youtube.com/watch?v=TGwZnZYg0jw

Profiling for Pandas:
https://github.com/pandas-profiling/pandas-profiling